In [13]:
import pandas as pd
import sqlalchemy
from pathlib import Path
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine

In [14]:
year_list = [2016, 2017, 2018, 2019, 2020, 2021]

def file_import(year):
    def file_reader_function(type, year):
        df = pd.read_csv(Path(f"../Resources/CRSS_data/{type}/{type}_{year}.csv"))
        return df

    accident_df = file_reader_function("accident", year)
    person_df = file_reader_function("person", year)
    vehicle_df = file_reader_function("vehicle", year)

    accident_df = accident_df[["CASENUM", "MAN_COLL", "WEATHER", "MONTH", "HOUR", "DAY_WEEK", "ALCOHOL"]]
    person_df = person_df[["CASENUM", "VEH_NO", "PER_NO", "AGE", "SEX", "DRUGS", "INJ_SEV"]]
    vehicle_df = vehicle_df[["CASENUM", "VEH_NO", "BODY_TYP", "SPEEDREL" ]]

    # Join Vehicle and Person DataFrames to create "veh_and_per_df", left join to Vehicle DF. (Written by Ed)
    veh_and_per_df = pd.merge(vehicle_df, person_df, on=['CASENUM','VEH_NO'], how='left')

    # Join veh_and_per_df and accident df to create "allinfo_df", left join to veh_and_per_df. (Written by Ed)
    data_df = pd.merge(veh_and_per_df,accident_df, on=['CASENUM'], how='left')

    data_df = data_df[(data_df["PER_NO"] == 1)]
    data_df = data_df.dropna(how="any")
    data_df = data_df.drop(columns=["CASENUM", "VEH_NO", "PER_NO"])

    def drop_index_function(column_list, value_list):
        for columns in column_list:
            for values in value_list:
                drop_index = data_df[(data_df[columns] == values)].index
                data_df.drop(drop_index, inplace=True)

    first_list = ['SEX', 'DAY_WEEK', 'INJ_SEV']
    first_values = [8, 9]

    drop_index_function(first_list, first_values)

    second_list = ['MAN_COLL', 'WEATHER', 'HOUR']
    second_values = [98, 99]

    drop_index_function(second_list, second_values)

    third_list = ['AGE']
    third_values = [998, 999]

    drop_index_function(third_list, third_values)

    fourth_list = ['BODY_TYP']
    fourth_values = [9, 19, 29, 39, 48, 49, 59, 71, 73, 89, 98, 99]

    drop_index_function(fourth_list, fourth_values)

    def replace_function(original, new):
        for row in original:
            data_df.loc[data_df["WEATHER"] == row] = new


    weather_1 = [[3, 5, 6, 7], 8]
    weather_2 = [[11], 4]
    weather_3 = [[12], 2]

    replace_function(weather_1[0], weather_1[1])
    replace_function(weather_2[0], weather_2[1])
    replace_function(weather_3[0], weather_3[1])

    data_df["WEATHER"].unique()

    data_df = data_df.reset_index(drop=True)
    data_df["KEY"] = str(year) + data_df.index.astype(str)
    column_reorder = data_df.columns.tolist()
    column_reorder = column_reorder[-1:] + column_reorder[:-1]
    data_df = data_df[column_reorder]

    data_df["AGE"] = data_df["AGE"].astype(int)
    data_df["SEX"] = data_df["SEX"].astype(int)
    data_df["DRUGS"] = data_df["DRUGS"].astype(int)
    data_df["INJ_SEV"] = data_df["INJ_SEV"].astype(int)

    # Creating SQL Alchemy Engine and connection to engine
    database_path = ("FlaskApp/Accidents_Database.db")
    engine = create_engine(f"sqlite:///{database_path}")
    conn = engine.connect()

    data_df.to_sql("CRSS_data", conn, schema=None, if_exists='append', index=False, index_label=None, chunksize=None, dtype=None, method=None)

In [15]:
for year in year_list:
    file_import(year)

C:\Users\mirai\AppData\Local\Temp\ipykernel_13600\3713324323.py:5: DtypeWarning: Columns (23,24,70,71) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(Path(f"../Resources/CRSS_data/{type}/{type}_{year}.csv"))
C:\Users\mirai\AppData\Local\Temp\ipykernel_13600\3713324323.py:5: DtypeWarning: Columns (26,27,74) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(Path(f"../Resources/CRSS_data/{type}/{type}_{year}.csv"))
C:\Users\mirai\AppData\Local\Temp\ipykernel_13600\3713324323.py:5: DtypeWarning: Columns (26,27,74) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(Path(f"../Resources/CRSS_data/{type}/{type}_{year}.csv"))
C:\Users\mirai\AppData\Local\Temp\ipykernel_13600\3713324323.py:5: DtypeWarning: Columns (45,47,141) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(Path(f"../Resources/CRSS_data/{type}/{type}_{year}